In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('mode.chained_assignment', None)


In [ ]:
#https://www.kaggle.com/chicago/chicago-red-light-and-speed-camera-data
csv_path_locations= "Data_Files/speed-camera-locations.csv"
csv_path_violations="Data_Files/speed-camera-violations.csv"

#https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
csv_path_crashes="Data_Files/Traffic_Crashes_-_Crashes.csv"

camera_locations_df= pd.read_csv(csv_path_locations)
violations_df= pd.read_csv(csv_path_violations)
crashes_df= pd.read_csv(csv_path_crashes)
pd.set_option('max_columns', None)
violations_df


In [ ]:
#Filter out violation columns and pull out year 

violation_dates= violations_df[["CAMERA ID","VIOLATION DATE"]]
violation_dates['YEAR']= pd.DatetimeIndex(violation_dates["VIOLATION DATE"]).year
violation_dates['MONTH']= pd.DatetimeIndex(violation_dates["VIOLATION DATE"]).month
violation_dates['YEAR_MONTH']= pd.to_datetime(violation_dates[["YEAR","MONTH"]].assign(Day=1))

#bin the violations by quarter
min(violation_dates['YEAR_MONTH'])
max(violation_dates['YEAR_MONTH'])
quarterly_bins_violations= pd.date_range(start='2014-03-01',end='2020-03-01', freq="Q")
violations_labels= []
for x in range(1,len(quarterly_bins_violations)):
    violations_labels.append(str(quarterly_bins_violations[x]))
violation_dates["QUARTERLY BINS"]= pd.cut(violation_dates['YEAR_MONTH'], quarterly_bins_violations, labels=violations_labels)
violation_dates

#groupby
violation_dates_grouped= violation_dates.groupby("QUARTERLY BINS").count()
violation_dates_grouped






In [ ]:
#Filter out camera location columns and pull out year and month
camera_locations_df.head()
# crash_dates_df['YEAR']= pd.DatetimeIndex(crash_dates_df["CRASH_DATE"]).year
camera_install_dates= camera_locations_df[["GO-LIVE DATE","ADDRESS"]]
camera_install_dates['YEAR']= pd.DatetimeIndex(camera_install_dates["GO-LIVE DATE"]).year
camera_install_dates['MONTH']= pd.DatetimeIndex(camera_install_dates["GO-LIVE DATE"]).month
# violation_dates['YEAR_MONTH']= pd.to_datetime(violation_dates[["YEAR","MONTH"]].assign(Day=1))
camera_install_dates['YEAR_MONTH']=pd.to_datetime(camera_install_dates[['YEAR','MONTH']].assign(Day=1))
camera_install_dates.head()




#bin the violations by quarter
quarterly_bins_cameras= pd.date_range(start='2013-05-01',end='2020-03-01', freq="Q")
cameras_labels= []
for x in range(1, len(quarterly_bins_cameras)):
    cameras_labels.append(str(quarterly_bins_cameras[x]))

# violation_dates["QUARTER BINS"]= pd.cut(violation_dates['YEAR_MONTH'], quarterly_bins, labels=violations_labels)
# violation_dates  

camera_install_dates["QUARTERLY BINS"]=pd.cut(camera_install_dates['YEAR_MONTH'], 
                          quarterly_bins_cameras, labels=cameras_labels )


#df_na['sex'].value_counts(dropna=False
camera_install_dates["QUARTERLY BINS"].value_counts(dropna=False)


cameras_byQuarter= camera_install_dates.groupby("QUARTERLY BINS").count()
cameras_byQuarter


# add totals cameras at end of each year
quarters= cameras_byQuarter.index.tolist()
quarter_totals= []
total_installed= 0
for quarter in quarters:
    total_installed=  total_installed + cameras_byQuarter.loc[quarter, ["GO-LIVE DATE"]]
    quarter_totals.append(int(total_installed))
    
cameras_byQuarter["TOTAL LIVE"]= quarter_totals
cameras_byQuarter

In [ ]:
#Filter out crash date columns and pull out year and month

crash_dates_df= crashes_df[["CRASH_DATE","CRASH_MONTH","INJURIES_TOTAL"]]
crash_dates_df= crash_dates_df.rename(columns={"CRASH_MONTH":"MONTH"})
crash_dates_df.head()                           
pd.set_option('mode.chained_assignment', None)
crash_dates_df['YEAR']= pd.DatetimeIndex(crash_dates_df["CRASH_DATE"]).year
crash_dates_df['YEAR_MONTH']= pd.to_datetime(crash_dates_df[["YEAR","MONTH"]].assign(Day=1))

In [ ]:
#Bin crashes by quarters
quarterly_bins_crashes= pd.date_range(start='2014-01-01',end='2022-01-01', freq="Q")
crash_labels= []
for x in range(1,len(quarterly_bins_crashes)):
    crash_labels.append(str(quarterly_bins_crashes[x]))
crash_dates_df["QUARTERLY BINS"]= pd.cut(crash_dates_df['YEAR_MONTH'], quarterly_bins_crashes, labels=crash_labels)
crash_dates_df

In [ ]:
#group crashes by quarterly bins
crash_dates_grouped= crash_dates_df.groupby("QUARTERLY BINS").count()
crash_dates_grouped

Displaying All Tables

In [ ]:
violation_dates_grouped

In [ ]:
cameras_byQuarter

In [ ]:
crash_dates_grouped

In [ ]:

violations_cameras_merge= pd.merge(violation_dates_grouped,cameras_byQuarter, left_index=True, right_index=True, how="inner")
violations_cameras_merge


In [ ]:
#Filter down columns
violations_cameras_merge= violations_cameras_merge[["CAMERA ID","GO-LIVE DATE","TOTAL LIVE"]]
violations_cameras_merge

In [ ]:
#rename columns
violations_cameras_merge= violations_cameras_merge.rename(columns= {"CAMERA ID":"TOTAL VIOLATIONS",
                                                                    "GO-LIVE DATE":"CAMERAS ADDED",
                                                                   "TOTAL LIVE":"TOTAL CAMERAS LIVE"})

In [ ]:
#violations per camera column
violations_cameras_merge['VIOLATIONS PER CAMERA']=violations_cameras_merge['TOTAL VIOLATIONS']/violations_cameras_merge['TOTAL CAMERAS LIVE']
violation_cameras_clean= violations_cameras_merge
violation_cameras_clean

In [ ]:
violation_cameras_clean.plot.scatter(x='TOTAL CAMERAS LIVE' ,y='TOTAL VIOLATIONS')

In [ ]:
violation_cameras_clean.plot.scatter(x='TOTAL CAMERAS LIVE' ,y='VIOLATIONS PER CAMERA')

In [ ]:
violation_cameras_clean.plot.line(y='VIOLATIONS PER CAMERA', rot="45")

In [ ]:
cameras_and_crashes= pd.merge(crash_dates_grouped,cameras_byQuarter, left_index=True, right_index=True, how="inner")


In [ ]:
cameras_and_crashes= cameras_and_crashes[["CRASH_DATE","GO-LIVE DATE","TOTAL LIVE"]]
cameras_and_crashes= cameras_and_crashes.rename(columns={"CRASH_DATE":"TOTAL CRASHES",
                                                         "GO-LIVE DATE":"CAMERAS ADDED",
                                                         "TOTAL LIVE":"TOTAL CAMERAS LIVE"})

In [ ]:
cameras_and_crashes_clean= cameras_and_crashes
cameras_and_crashes_clean

In [ ]:
# output merged dataframes to CSVs



In [ ]:
cameras_and_crashes_clean.plot.scatter(x='TOTAL CAMERAS LIVE',y=
                                 'TOTAL CRASHES')



In [ ]:
cameras_and_crashes_clean.plot.line( y='TOTAL CRASHES', rot="45")

In [ ]:
# output merged dataframes to CSVs

cameras_and_crashes.to_csv('Output Data/cameras_and_crashes.csv', index=True)
cameras_and_crashes_clean.to_csv('Output Data/cameras_and_crashes_clean.csv', index=True)
violation_cameras_clean.to_csv('Output Data/violations_cameras_clean.csv', index=True)

#### PHASE ONE BEGINS HERE #####